<a href="https://colab.research.google.com/github/segunumoru1/-Predicting_Missing_Values_in_Environmental_Data/blob/master/Final_working_stage_II_Project_Test_(Used_to_extract_info_from_PDF)_LangChainWithFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Background**
The field of Generative AI Assistants for Government forms has been a subject of growing interest and scholarly investigation in recent years. As a need to address the challenge of assisting individuals, particularly those with disabilities such as the deaf, the blind, etc., in completing government forms like the Adult Disability Payment Form. Understanding the various perspectives and developments within this domain has become increasingly crucial.

## Problem Statetment
Develop an AI Assistant Chatbot in filling Government Disability Forms

In [1]:
# pip install all the necessary dependencies
!pip install python-dotenv
!pip install requests
!pip install langchain==0.1.0
!pip install streamlit
!pip install openai
!pip install pdfplumber
!pip install PyMuPDF  # PyMuPDF
!pip install pyngrok
!pip install tiktoken
!pip install faiss-gpu
!pip install faiss-cpu
!pip install python-docx python-pptx
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow docx2txt

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtim

### **Core Libraries**

*openai:* The official OpenAI Python library, used to interact with OpenAI's models like GPT-3.5 and GPT-4 for text generation, translation, question answering, and more.

*requests:* A popular library for making HTTP requests to web services and APIs, allowing you to fetch data from the internet.

*os:* Python's built-in library for interacting with the operating system, providing functionalities like file system navigation and environment variable management.

*datetime:* Python's built-in library for handling dates and times, useful for scheduling tasks, logging events, and working with time-sensitive data.

*pdfplumber*: This is a Python library for extracting text and other information from PDF files.

*PyMuPDF (fitz)*: This is a Python binding for the MuPDF library, which is used for working with PDF and other document formats.

*pyngrok*: This is a Python library that allows you to expose a local web server to the internet using the ngrok service.

*tiktoken*: This is a Python library for fast encoding and decoding of text using the Byte Pair Encoding (BPE) algorithm.

*faiss-gpu and faiss-cpu*: These are Python libraries for efficient similarity search and clustering of dense vectors, with GPU and CPU support respectively.

*python-docx and python-pptx*: These are Python libraries for creating and manipulating Microsoft Word (DOCX) and PowerPoint (PPTX) documents.

*tesseract-ocr*: This is an open-source Optical Character Recognition (OCR) engine that can be used to extract text from images.

*pytesseract*: This is a Python wrapper for Google's Tesseract-OCR Engine, which allows you to use Tesseract-OCR from within your Python code.

*pillow*: This is a Python Imaging Library (PIL) that provides image processing capabilities.

*docx2txt*: This is a Python library that can extract text from Microsoft Word (DOCX) documents.

**LangChain**

*langchain.chat_models.ChatOpenAI*: Provides a wrapper for interacting with OpenAI's chat models in a more structured way, facilitating the creation of conversational agents.

*langchain.chains.LLMChain:* A core component of LangChain that allows you to chain together language models and other components to create more complex workflows.

*langchain.memory.ConversationBufferWindowMemory*: A type of memory that stores a history of the conversation, allowing the language model to maintain context and provide more relevant responses.

*langchain.prompts.PromptTemplate:* A tool for creating and managing prompts for language models, making it easier to structure and customize your interactions with the model.
Other

*streamlit:* A framework for building interactive web applications for data science and machine learning, allowing you to easily create user interfaces for your models and data.

*python-dotenv:* A library for loading environment variables from a .env file, helping you manage sensitive information like API keys securely.

In [2]:
import os
import openai
from google.colab import userdata

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
# Get the API key from the environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
# import the necessary libraries
import fitz  # PyMuPDF
import json
import requests
import time
import datetime
import warnings
import tiktoken
import faiss
import fitz  # PyMuPDF
from docx import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.schema import SystemMessage
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    StringPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks import get_openai_callback
import fitz  # PyMuPDF
from docx import Document
import docx2txt
import io
from PIL import Image
import pytesseract
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

This is the LangChain Bot version with document access for Context Augmentation.
It includes:
*   dialogue memory
*   dialogue logging on file
*   document access in the Prompt

CAUTION: large pdf documents tend to slow down the ChatBot and cost more tokens. A reasonable size is up to 5 pages.







In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Referrral Form 03.pdf to Referrral Form 03.pdf


1. from google.colab import files: This line imports the files module from the google.colab library. This module provides functions for interacting with the Colab file system, including uploading and downloading files.

2. uploaded = files.upload(): This line calls the upload() function from the files module. When executed, it displays a button in the Colab notebook that allows the user to select files from their local machine to upload. The uploaded files are stored in the Colab environment and a dictionary is returned, where keys are the file names and values are the file

In [ ]:
# PDF Extraction Class objects and method for processing and vector embeddings

class DocumentProcessor:
    def __init__(self):
        self.text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        self.embeddings = OpenAIEmbeddings()
        self.vectorstore = None

    def process_pdf(self, pdf_path):
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()

            # Extract images
            images = page.get_images(full=True)
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]

                # Use PIL to open the image
                image = Image.open(io.BytesIO(image_bytes))

                # Perform OCR on the image
                image_text = pytesseract.image_to_string(image)
                text += f"\nImage {img_index + 1} text:\n{image_text}\n"

        return text

    def process_docx(self, docx_path):
        # Extract text
        text = docx2txt.process(docx_path)

        # Extract images
        temp_dir = "temp_images"
        os.makedirs(temp_dir, exist_ok=True)
        docx2txt.process(docx_path, temp_dir)

        # Perform OCR on extracted images
        for img_file in os.listdir(temp_dir):
            if img_file.endswith(('png', 'jpg', 'jpeg', 'tiff')):
                img_path = os.path.join(temp_dir, img_file)
                image_text = self.extract_text_from_image(img_path)
                text += f"\nImage {img_file} text:\n{image_text}\n"
                os.remove(img_path)  # Clean up the temporary image file

        os.rmdir(temp_dir)  # Remove the temporary directory
        return text

    def extract_text_from_image(self, image_path):
        image = Image.open(image_path)
        text = pytesseract.image_to_string(image)
        return text

    def process_file(self, file_path):
        _, file_extension = os.path.splitext(file_path)
        if file_extension.lower() == '.pdf':
            return self.process_pdf(file_path)
        elif file_extension.lower() in ['.docx', '.doc']:
            return self.process_docx(file_path)
        else:
            raise ValueError(f"Unsupported file type: {file_extension}")

    def add_document(self, file_path):
        text = self.process_file(file_path)
        texts = self.text_splitter.split_text(text)

        if self.vectorstore is None:
            self.vectorstore = FAISS.from_texts(texts, self.embeddings)
        else:
            self.vectorstore.add_texts(texts)

    def search(self, query, k=3):
        if self.vectorstore is None:
            return []
        return self.vectorstore.similarity_search(query, k=k)

In [ ]:
# Usage example:
processor = DocumentProcessor()
processor.add_document('/content/drive/MyDrive/Datasets/PDFs_DB/Referrral Form 01.pdf')
processor.add_document('/content/drive/MyDrive/Datasets/PDFs_DB/Referrral Form 02.pdf')
processor.add_document('/content/drive/MyDrive/Datasets/PDFs_DB/Referrral Form 03.pdf')
results = processor.search("personal details")
print(results)

[Document(page_content='Image 1 text:\n  \n  \n\n  \n\nPERSONAL DETAILS (cont.)\n\nLiving Arrangements:\n\n \n         \n     \n   \n \n    \n      \n  \n   \n\nCarer role in household Living with parents/guardian\n\nCaring for Children Living with partner\nLiving alone Living with other relatives/friends\n\nLiving in homeless unit Looked after at home\n\na. o-oo @\n\nLiving in residential/secure accommodation Other (please specify)\n\nae: ee oe\n\nLiving in supported accommodation\n\n \n   \n\nLiving with foster CTO => «= (titi eine ae eta MD ain seas naan Nes kd Aen wem nena seevencnsecensonane ness\n\nGP Telephone No:\nActual Name of Practice: CHI Number\n\nDoes the person have any medical/mental health conditions? O YES 0 NO\nPlease give details :\n\n  \n\n|\nxy\n\n  \n\n  \n\nIs the person taking any form of medication? O YES LINO\n\nIf so please indicate what type:\n\nREFERRAL DETAILS\nReferrer: Relationship to service user:\nAddress:\n\nPostcode:\n\nIs the person aware of the se

In [ ]:
# Disability Form Assistant class

class DisabilityFormAssistant:
    def __init__(self, pdf_paths, model="gpt-4o", temperature=0.7):
        self.api_key = os.getenv("OPENAI_API_KEY")
        openai.api_key = self.api_key
        self.model = model
        self.temperature = temperature
        self.llm = self._setup_llm()
        self.document_processor = DocumentProcessor()
        self.memory = ConversationBufferWindowMemory(memory_key="history", input_key="human_input")
        self.chat_chain = self._setup_chat_chain()

        for pdf_path in pdf_paths:
            self.add_document(pdf_path)

    def _setup_llm(self):
        return ChatOpenAI(model=self.model, temperature=self.temperature)

    def _setup_chat_chain(self):
        template = """
        Act as if you are a professional assistant specialized in helping people with disabilities fill out official documentation.\
        Never ever mention that you are an AI or a language model.\
        You provide step-by-step guidance to fill out forms accurately.\
        You can answer questions about the entire process.\

        Take all procedure from the website https://www.mygov.scot/adult-disability-payment\
        Break down complex instructions into simple, manageable steps.\

        In your answer, write each sentence in one line to aid reading.\
        Include specific references to sections and pages of the documents when applicable.\

        If you cannot find the answer in the material, then state that the answer is not in the document.\

        This is your prompt.

        Context from the document:
        {context}

        {history}
        Human: {human_input}
        Assistant:
        """
        prompt = PromptTemplate(input_variables=["context", "history", "human_input"], template=template)
        return LLMChain(llm=self.llm, prompt=prompt, memory=self.memory)

    def add_document(self, file_path):
        self.document_processor.add_document(file_path)

    def get_response(self, query):
        start_time = time.time()

        # Retrieve relevant context from the vectorstore
        docs = self.document_processor.search(query, k=3)
        context = "\n".join([doc.page_content for doc in docs]) if docs else ""

        # Generate response using the chat chain
        response = self.chat_chain.predict(human_input=query, context=context)

        end_time = time.time()
        response_time = end_time - start_time
        return response, response_time

    def get_user_rating(self):
        while True:
            try:
                rating = int(input("Please rate your experience (1-5): "))
                if 1 <= rating <= 5:
                    return rating
                else:
                    print("Please enter a number between 1 and 5.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def save_dialogue(self, dialogue):
        timestamp = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
        filename = f"Dialogue_{timestamp}.txt"
        with open(filename, "w") as file:
            file.write(dialogue)
        print(f"Dialogue saved to {filename}")

In [ ]:
def main():
    pdf_paths = ["/content/drive/MyDrive/Datasets/PDFs_DB/Referrral Form 01.pdf",
                 "/content/drive/MyDrive/Datasets/PDFs_DB/Referrral Form 02.pdf",
                 "/content/drive/MyDrive/Datasets/PDFs_DB/Referrral Form 03.pdf"]  # Add your PDF paths here
    assistant = DisabilityFormAssistant(pdf_paths)
    dialogue = ""

    print("Welcome! I'm here to assist you with the Adult Disability Payment form.")
    print("Type 'exit' to end the conversation.")
    print("You can also type 'add document' to add a new document to the knowledge base.")

    while True:
        user_input = input("User > ")
        dialogue += f"User: {user_input}\n"

        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'add document':
            file_path = input("Enter the path to the document: ")
            assistant.add_document(file_path)
            print("Document added successfully!")
            dialogue += f"Document added: {file_path}\n"
            continue

        response, response_time = assistant.get_response(user_input)
        print(f"Assistant: {response}")
        print(f"Response time: {response_time:.2f} seconds")

        dialogue += f"Assistant: {response}\n"
        dialogue += f"Response time: {response_time:.2f} seconds\n\n"

    rating = assistant.get_user_rating()
    dialogue += f"User rating: {rating}/5\n"

    assistant.save_dialogue(dialogue)
    print(f"Thank you for your rating: {rating}/5")
    print("Thank you for using the Disability Form Assistant. Goodbye!")

if __name__ == "__main__":
    main()

Welcome! I'm here to assist you with the Adult Disability Payment form.
Type 'exit' to end the conversation.
You can also type 'add document' to add a new document to the knowledge base.
User > Hello.  can you help me with filling my form?
Assistant: Of course, I'd be happy to help you fill out your form. Let's begin by going through each section step-by-step. Please have your form in front of you, and let's start with the "Personal Details" section.

### Personal Details
1. **Living Arrangements:**
   - Select the option that best describes your current living situation. Options include:
     - Living with parents/guardian
     - Living with partner
     - Living alone
     - Living with other relatives/friends
     - Living in a homeless unit
     - Looked after at home
     - Living in residential/secure accommodation
     - Living in supported accommodation
     - Living with foster care
     - Other (please specify)
   
2. **GP Telephone No:**
   - Enter the telephone number of yo

KeyboardInterrupt: Interrupted by user

# Break down:
1. I have added the necessary libraries for PDF processing and vector storage.

2. The process_pdf function extracts text from the PDF, splits it into chunks, and creates a vector store for efficient searching.

3. The DisabilityFormAssistant class now takes a pdf_path parameter and uses the processed PDF as the primary source of information.

4. The get_response method now retrieves relevant context from the vector store before generating a response.
The prompt template has been updated to include the context from the PDF.